# Data Preparation for the New Entity Matching Benckmark

Following extensive research and conscientious analysis, we have developed a new benchmark with respect to the entity matching task, specifically tailored for approaches with foundation models. This benchmark encompasses datasets from five diverse domains: restaurant, book, paper, movie, and product. Each domain features three unique datasets.

P.S. All the source datasets are obtained from the [Magellan Data Repository](https://sites.google.com/site/anhaidgroup/useful-stuff/the-magellan-data-repository). This notebook contains all the necessary code to construct the benchmark.

In [2]:
# import necessary libraries
import requests
import tarfile
import os

import numpy as np
import pandas as pd
import duckdb
from sklearn.feature_extraction.text import TfidfVectorizer

np.random.seed(42)
SEED = 42

## Download Source Datasets to Local

The initial step involves downloading all the source datasets from the Magellan repository. Since these datasets are provided in a compressed format, we will decompress them and only retain the decompressed versions locally.

In [16]:
urls = {
    'restaurant': ['http://pages.cs.wisc.edu/~anhai/data/784_data/restaurants2.tar.gz', 
                   'http://pages.cs.wisc.edu/~anhai/data/784_data/restaurants4.tar.gz', 
                   'http://pages.cs.wisc.edu/~anhai/data/corleone_data/restaurants.tar.gz'],
    'book': ['http://pages.cs.wisc.edu/~anhai/data/784_data/books2.tar.gz', 
             'http://pages.cs.wisc.edu/~anhai/data/784_data/books3.tar.gz', 
             'http://pages.cs.wisc.edu/~anhai/data/784_data/books5.tar.gz'],
    'paper': ['http://pages.cs.wisc.edu/~anhai/data/corleone_data/citations.tar.gz', 
              'http://pages.cs.wisc.edu/~anhai/data/wisc_em_benchmark/839_spring19/CompVision/csv/CompVision.tar.gz', 
              'http://pages.cs.wisc.edu/~anhai/data/wisc_em_benchmark/839_spring19/AcadPapers/csv/AcadPapers.tar.gz'],
    'movie': ['http://pages.cs.wisc.edu/~anhai/data/784_data/movies1.tar.gz', 
              'http://pages.cs.wisc.edu/~anhai/data/784_data/movies4.tar.gz', 
              'http://pages.cs.wisc.edu/~anhai/data/784_data/movies5.tar.gz'],
    'product': ['http://pages.cs.wisc.edu/~anhai/data/784_data/cosmetics.tar.gz', 
                'http://pages.cs.wisc.edu/~anhai/data/784_data/baby_products.tar.gz', 
                'http://pages.cs.wisc.edu/~anhai/data/corleone_data/products.tar.gz']
}

In [17]:
def download_and_extract(url, extract_dir='.'):
    """
    Download a .tar.gz file from a specified URL and extract its contents.
    """
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open("temp_file.tar.gz", 'wb') as file:
            file.write(response.raw.read())
        
        with tarfile.open("temp_file.tar.gz", "r:gz") as tar_ref:
            tar_ref.extractall(path=extract_dir)
        
        os.remove("temp_file.tar.gz")
    else:
        print(f"Error downloading the file: HTTP {response.status_code}")
        
        
# [download_and_extract(url, os.path.join('dirty', domain)) for domain in urls for url in urls[domain]]

## Restaurant Domain

In [18]:
src_restaurant_dir_path = 'dirty/restaurant'

### Fodors & Zagats
The dataset collector did not specify Table A and Table B. Therefore, in alignment with the practices of existing benchmarks, we have named the final dataset 'Fodors_Zagats.' A separate file has been designated to store the matching pairs annotated by humans.

Metadata of the source data: The Fodor dataset comprises 533 samples, while the Zagat dataset contains 331 samples. Human annotators have labeled 112 pairs as gold-standard matches. Additionally, there are 82 pairs that exhibit an exact match at the string level in the 'name' column. 

The following are details to generate postive candidates (matches) and negative candidates (non-matches):

**postive candidates** : 
1. The nameA contains nameB, vice verse; OR
2. The addressA contains addressB, vice verse; OR
3. The phoneA contains phoneB, vice verse; OR
4. The levenshtein similarity between nameA and nameB is larger than 0.9.

**negative candidates** :
1. The idA and idB is distinct from the positive candidates; AND
2. The idA and idB is distinct from the golden candidates.
3. 1000 pairs are randonly selected afterward.

The ['Name', 'Address', 'Phone'] columns are used for the later manual check.

In [19]:
dataset_path = 'restaurants'
tableA_path = os.path.join(src_restaurant_dir_path, dataset_path, 'fodors.csv')
tableB_path = os.path.join(src_restaurant_dir_path, dataset_path, 'zagats.csv')

tableA = pd.read_csv(tableA_path)
tableB = pd.read_csv(tableB_path)

tableA.replace(r'^\s*$', np.nan, regex=True, inplace=True)
tableB.replace(r'^\s*$', np.nan, regex=True, inplace=True)

match_path = os.path.join(src_restaurant_dir_path, dataset_path, 'matches_fodors_zagats.csv')
gold_match = pd.read_csv(match_path)

print(len(set(tableA.name).intersection(set(tableB.name))))
print(len(gold_match))

83
112


In [20]:
# clean up
print('The old columns are: \n', tableA.columns, '\n', tableB.columns)

new_tableA = tableA.drop(columns=['class']).rename(columns={'id': 'ID', 'name': 'Name', 'addr': 'Address', 
                                                            'city': 'City', 'phone': 'Phone'})
new_tableB = tableB.drop(columns=['class']).rename(columns={'id': 'ID', 'name': 'Name', 'addr': 'Address', 
                                                            'city': 'City', 'phone': 'Phone'})

print('\n The new columns are: \n', new_tableA.columns, '\n', new_tableB.columns)


# store new_tableA and new_tableB in the cleaned directory
save_dir = 'cleaned/restaurant/Fodors-Zagats'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
new_tableA.to_csv(os.path.join(save_dir, 'tableA.csv'), index=False)
new_tableB.to_csv(os.path.join(save_dir, 'tableB.csv'), index=False)

The old columns are: 
 Index(['id', 'name', 'addr', 'city', 'phone', 'type', 'class'], dtype='object') 
 Index(['id', 'name', 'addr', 'city', 'phone', 'type', 'class'], dtype='object')

 The new columns are: 
 Index(['ID', 'Name', 'Address', 'City', 'Phone', 'type'], dtype='object') 
 Index(['ID', 'Name', 'Address', 'City', 'Phone', 'type'], dtype='object')


In [21]:
temp_tableA = new_tableA[['ID', 'Name', 'Address', 'Phone']].copy()
temp_tableB = new_tableB[['ID', 'Name', 'Address', 'Phone']].copy()

temp_tableA['_Phone'] = temp_tableA['Phone'].str.replace('/', '-')
temp_tableA['_Address'] = temp_tableA['Address'].apply(lambda x: 'This is not an address' if x == '4' else x)

connection = duckdb.connect(database=':memory:', read_only=False)
connection.register('A', temp_tableA)
connection.register('B', temp_tableB)

query = """
SELECT
    ta.ID AS ID_A,
    tb.ID AS ID_B,
    ta.Name as Name_A,
    tb.Name as Name_B,
    ta.Address as Address_A,
    tb.Address as Address_B,
    ta.Phone as Phone_A,
    tb.Phone as Phone_B
    
FROM
    A ta
CROSS JOIN
    B tb
WHERE
    (ta.Name LIKE '%' || tb.Name || '%' OR tb.Name LIKE '%' || ta.Name || '%') OR
    (ta._Address LIKE '%' || tb.Address || '%' OR tb.Address LIKE '%' || ta._Address || '%') OR
    (ta._Phone LIKE '%' || tb.Phone || '%' OR tb.Phone LIKE '%' || ta._Phone || '%') OR
    (levenshtein(ta.Name, tb.Name) / GREATEST(LENGTH(ta.Name), LENGTH(tb.Name)) < 0.1)
"""

result = connection.execute(query).fetchdf()
connection.close()

In [22]:
## generate pos_candidates & neg_candidates, and store them in the cleaned directory
pos_candidates = pd.DataFrame({
    'ID_A': result['ID_A'], 'ID_B': result['ID_B'],
    'Name_A': result['Name_A'], 'Name_B': result['Name_B'],
    'Address_A': result['Address_A'], 'Address_B': result['Address_B'],
    'Phone_A': result['Phone_A'], 'Phone_B': result['Phone_B'],
})

temp_tableA['temp_key'] = 1
temp_tableB['temp_key'] = 1
all_id_pairs = pd.merge(temp_tableA[['ID', 'temp_key']], temp_tableB[['ID', 'temp_key']], on='temp_key', suffixes=('_A', '_B'))
all_id_pairs = all_id_pairs.drop('temp_key', axis=1)


gold_match = gold_match.rename(columns={'fodors_id': 'ID_A', 'zagats_id': 'ID_B'})
exclude_pairs = pd.concat([gold_match, pos_candidates[['ID_A', 'ID_B']]], axis=0).drop_duplicates()
neg_candidates = pd.concat([all_id_pairs, exclude_pairs], axis=0).drop_duplicates(keep=False)
neg_candidates = neg_candidates.sample(n=1000, random_state=SEED)

neg_candidates = neg_candidates.merge(temp_tableA[['ID', 'Name', 'Address', 'Phone']], 
                                      left_on='ID_A', right_on='ID', how='left', suffixes=('_A', '_B')).drop(columns=['ID'])
neg_candidates = neg_candidates.merge(temp_tableB[['ID', 'Name', 'Address', 'Phone']], 
                                      left_on='ID_B', right_on='ID', how='left', suffixes=('_A', '_B')).drop(columns=['ID'])
neg_candidates = neg_candidates[list(pos_candidates.columns)]

pos_candidates.to_csv(os.path.join(save_dir, 'pos_candidates.csv'), index=False)
neg_candidates.to_csv(os.path.join(save_dir, 'neg_candidates.csv'), index=False)

### Restaurant 2

The dataset consist of tableA from Zomato and tableB from Yelp.

Metadata of the source data: The Zomato dataset comprises 6960 samples, while the Yelp dataset contains 3897 samples. Human annotators have labeled 90 pairs as gold-standard matches. Additionally, there are 1042 pairs that exhibit an exact match at the string level in the 'name' column.


The following are details to generate postive candidates (matches) and negative candidates (non-matches):

**postive candidates** : 
1. The nameA contains nameB, vice verse; AND
2. The zipcodeA and zipcodeB are the same.

**negative candidates** :
1. The idA and idB is distinct from the positive candidates; AND
2. The idA and idB is distinct from the golden candidates.
3. 1000 pairs are randonly selected afterward.

The ['Name', 'Zipcode', 'Address'] columns are used for the later manual check.

In [23]:
dataset_path = 'restaurants2/csv_files'
tableA_path = os.path.join(src_restaurant_dir_path, dataset_path, 'zomato.csv')
tableB_path = os.path.join(src_restaurant_dir_path, dataset_path, 'yelp.csv')

tableA = pd.read_csv(tableA_path)
tableB = pd.read_csv(tableB_path)

tableA.replace(r'^\s*$', np.nan, regex=True, inplace=True)
tableB.replace(r'^\s*$', np.nan, regex=True, inplace=True)

match_path = os.path.join(src_restaurant_dir_path, dataset_path, 'labeled_data.csv')
gold_match = pd.read_csv(match_path, skiprows=5)

print(len(set(tableA.name).intersection(set(tableB.name))))
print(gold_match.gold.sum())

1042
90


In [24]:
# clean up
print('The old columns are: \n', tableA.columns, '\n', tableB.columns)

tableA['cuisine'] = tableA['cuisine'].apply(lambda x: x.replace(',', ', ') if not pd.isnull(x) else x)

new_tableA = tableA.drop(columns=['reviewcount']).rename(columns={'name': 'Name', 'votes':'Votes', 
                                                              'rating': 'Rating', 'phone': 'Phone', 
                                                              'address': 'Address', 'zip': 'Zipcode', 
                                                              'cuisine' : 'Cuisine'})
new_tableB = tableB.rename(columns={'name': 'Name', 'votes':'Votes', 
                                         'rating': 'Rating', 'phone': 'Phone', 
                                         'address': 'Address', 'zip': 'Zipcode', 
                                         'cuisine' : 'Cuisine'})

print('\n The new columns are: \n', new_tableA.columns, '\n', new_tableB.columns)

# store new_tableA and new_tableB in the cleaned directory
save_dir = 'cleaned/restaurant/Zomato-Yelp'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
new_tableA.to_csv(os.path.join(save_dir, 'tableA.csv'), index=False)
new_tableB.to_csv(os.path.join(save_dir, 'tableB.csv'), index=False)

The old columns are: 
 Index(['ID', 'name', 'votes', 'rating', 'phone', 'address', 'zip', 'cuisine',
       'reviewcount'],
      dtype='object') 
 Index(['ID', 'name', 'votes', 'rating', 'phone', 'address', 'zip', 'cuisine'], dtype='object')

 The new columns are: 
 Index(['ID', 'Name', 'Votes', 'Rating', 'Phone', 'Address', 'Zipcode',
       'Cuisine'],
      dtype='object') 
 Index(['ID', 'Name', 'Votes', 'Rating', 'Phone', 'Address', 'Zipcode',
       'Cuisine'],
      dtype='object')


In [25]:
temp_tableA = new_tableA[['ID', 'Name', 'Zipcode', 'Address']].copy()
temp_tableB = new_tableB[['ID', 'Name', 'Zipcode', 'Address']].copy()

connection = duckdb.connect(database=':memory:', read_only=False)
connection.register('A', temp_tableA)
connection.register('B', temp_tableB)

query = """
SELECT
    ta.ID AS ID_A,
    tb.ID AS ID_B,
    ta.Name AS Name_A,
    tb.Name AS Name_B,
    ta.Zipcode As Zipcode_A,
    tb.Zipcode As Zipcode_B,
    ta.Address AS Address_A,
    tb.Address AS Address_B,
    
    
FROM
    A ta
CROSS JOIN
    B tb
WHERE
    (LOWER(ta.Name) LIKE '%' || LOWER(tb.Name) || '%' OR LOWER(tb.Name) LIKE '%' || LOWER(ta.Name) || '%') AND
    (ta.Zipcode == tb.Zipcode)
"""

result = connection.execute(query).fetchdf()
connection.close()

In [26]:
## generate pos_candidates & neg_candidates, and store them in the cleaned directory
pos_candidates = pd.DataFrame({
    'ID_A': result['ID_A'], 'ID_B': result['ID_B'],
    'Name_A': result['Name_A'], 'Name_B': result['Name_B'],
    'Zipcode_A': result['Zipcode_A'], 'Zipcode_B': result['Zipcode_B'],
    'Address_A': result['Address_A'], 'Address_B': result['Address_B'],
})

temp_tableA['temp_key'] = 1
temp_tableB['temp_key'] = 1
all_id_pairs = pd.merge(temp_tableA[['ID', 'temp_key']], temp_tableB[['ID', 'temp_key']], on='temp_key', suffixes=('_A', '_B'))
all_id_pairs = all_id_pairs.drop('temp_key', axis=1)


gold_match = gold_match.rename(columns={'ltable.id': 'ID_A', 'rtable.id': 'ID_B'})
exclude_pairs = pd.concat([gold_match[['ID_A', 'ID_B']], pos_candidates[['ID_A', 'ID_B']]], axis=0).drop_duplicates()
neg_candidates = pd.concat([all_id_pairs, exclude_pairs], axis=0).drop_duplicates(keep=False)
neg_candidates = neg_candidates.sample(n=1000, random_state=SEED)

neg_candidates = neg_candidates.merge(temp_tableA[['ID', 'Name', 'Zipcode', 'Address']], 
                                      left_on='ID_A', right_on='ID', how='left', suffixes=('_A', '_B')).drop(columns=['ID'])
neg_candidates = neg_candidates.merge(temp_tableB[['ID', 'Name', 'Zipcode', 'Address']], 
                                      left_on='ID_B', right_on='ID', how='left', suffixes=('_A', '_B')).drop(columns=['ID'])
neg_candidates = neg_candidates[list(pos_candidates.columns)]

pos_candidates.to_csv(os.path.join(save_dir, 'pos_candidates.csv'), index=False)
neg_candidates.to_csv(os.path.join(save_dir, 'neg_candidates.csv'), index=False)

### Restaurants 4

The dataset consist of tableA from Yellow Pages and tableB from Yelp.

Metadata of the source data: The Yellow Pages dataset comprises 11840 samples, while the Yelp dataset contains 5223 samples. Human annotators have labeled 130 pairs as gold-standard matches. Additionally, there are 575 pairs that exhibit an exact match at the string level in the 'name' column.


The following are details to generate postive candidates (matches) and negative candidates (non-matches):

**postive candidates** : 
1. The nameA contains nameB, vice verse; AND
2. The zipcodeA and zipcodeB are the same.
3. 1100 pairs are randonly selected afterward.

**negative candidates** :
1. The idA and idB is distinct from the positive candidates; AND
2. The idA and idB is distinct from the golden candidates.
3. 1000 pairs are randonly selected afterward.

The ['Name', 'Zipcode', 'Address'] columns are used for the later manual check.

In [27]:
dataset_path = 'restaurants4/csv_files'
tableA_path = os.path.join(src_restaurant_dir_path, dataset_path, 'yellow_pages.csv')
tableB_path = os.path.join(src_restaurant_dir_path, dataset_path, 'yelp.csv')

tableA = pd.read_csv(tableA_path)
tableB = pd.read_csv(tableB_path)

tableA.replace(r'^\s*$', np.nan, regex=True, inplace=True)
tableB.replace(r'^\s*$', np.nan, regex=True, inplace=True)

match_path = os.path.join(src_restaurant_dir_path, dataset_path, 'labeled_data.csv')
gold_match = pd.read_csv(match_path, skiprows=5)

print(len(set(tableA.name).intersection(set(tableB.name))))
print(gold_match.gold.sum())

575
130


In [28]:
# clean up
print('The old columns are: \n', tableA.columns, '\n', tableB.columns)

new_tableA = tableA.drop(columns=['website', 'Unnamed: 8'])\
                   .rename(columns={'id': 'ID', 'name': 'Name', 'address': 'Address', 'city': 'City', 
                                    'state': 'State', 'zipcode': 'Zipcode', 'phone': 'Phone'})
new_tableB = tableB.drop(columns=[])\
                   .rename(columns={'id': 'ID', 'name': 'Name', 'address': 'Address', 'city': 'City', 
                                    'state': 'State', 'zipcode': 'Zipcode', 'phone': 'Phone'})

print('\n The new columns are: \n', new_tableA.columns, '\n', new_tableB.columns)

# store new_tableA and new_tableB in the cleaned directory
save_dir = 'cleaned/restaurant/Yellow_Pages-Yelp'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
new_tableA.to_csv(os.path.join(save_dir, 'tableA.csv'), index=False)
new_tableB.to_csv(os.path.join(save_dir, 'tableB.csv'), index=False)

The old columns are: 
 Index(['id', 'name', 'address', 'city', 'state', 'zipcode', 'phone', 'website',
       'Unnamed: 8'],
      dtype='object') 
 Index(['id', 'name', 'address', 'city', 'state', 'zipcode', 'phone'], dtype='object')

 The new columns are: 
 Index(['ID', 'Name', 'Address', 'City', 'State', 'Zipcode', 'Phone'], dtype='object') 
 Index(['ID', 'Name', 'Address', 'City', 'State', 'Zipcode', 'Phone'], dtype='object')


In [29]:
temp_tableA = new_tableA[['ID', 'Name', 'Zipcode', 'Address']].copy()
temp_tableB = new_tableB[['ID', 'Name', 'Zipcode', 'Address']].copy()

connection = duckdb.connect(database=':memory:', read_only=False)
connection.register('A', temp_tableA)
connection.register('B', temp_tableB)

query = """
SELECT
    ta.ID AS ID_A,
    tb.ID AS ID_B,
    ta.Name AS Name_A,
    tb.Name AS Name_B,
    ta.Zipcode As Zipcode_A,
    tb.Zipcode As Zipcode_B,
    ta.Address AS Address_A,
    tb.Address AS Address_B,
    
FROM
    A ta
CROSS JOIN
    B tb
WHERE
    (LOWER(ta.Name) LIKE '%' || LOWER(tb.Name) || '%' OR LOWER(tb.Name) LIKE '%' || LOWER(ta.Name) || '%') AND
    (ta.Zipcode == tb.Zipcode)
"""

result = connection.execute(query).fetchdf()
connection.close()

In [30]:
## generate pos_candidates & neg_candidates, and store them in the cleaned directory
pos_candidates = pd.DataFrame({
    'ID_A': result['ID_A'], 'ID_B': result['ID_B'],
    'Name_A': result['Name_A'], 'Name_B': result['Name_B'],
    'Zipcode_A': result['Zipcode_A'], 'Zipcode_B': result['Zipcode_B'],
    'Address_A': result['Address_A'], 'Address_B': result['Address_B'],
})

temp_tableA['temp_key'] = 1
temp_tableB['temp_key'] = 1
all_id_pairs = pd.merge(temp_tableA[['ID', 'temp_key']], temp_tableB[['ID', 'temp_key']], on='temp_key', suffixes=('_A', '_B'))
all_id_pairs = all_id_pairs.drop('temp_key', axis=1)

gold_match = gold_match.rename(columns={'ltable.id': 'ID_A', 'rtable.id': 'ID_B'})
exclude_pairs = pd.concat([gold_match[['ID_A', 'ID_B']], pos_candidates[['ID_A', 'ID_B']]], axis=0).drop_duplicates()
neg_candidates = pd.concat([all_id_pairs, exclude_pairs], axis=0).drop_duplicates(keep=False)

pos_candidates = pos_candidates.sample(n=1100, random_state=SEED)

neg_candidates = neg_candidates.sample(n=1000, random_state=SEED)
neg_candidates = neg_candidates.merge(temp_tableA[['ID', 'Name', 'Zipcode', 'Address']], 
                                      left_on='ID_A', right_on='ID', how='left', suffixes=('_A', '_B')).drop(columns=['ID'])
neg_candidates = neg_candidates.merge(temp_tableB[['ID', 'Name', 'Zipcode', 'Address']], 
                                      left_on='ID_B', right_on='ID', how='left', suffixes=('_A', '_B')).drop(columns=['ID'])
neg_candidates = neg_candidates[list(pos_candidates.columns)]

pos_candidates.to_csv(os.path.join(save_dir, 'pos_candidates.csv'), index=False)
neg_candidates.to_csv(os.path.join(save_dir, 'neg_candidates.csv'), index=False)

## Book Domain

In [16]:
src_book_dir_path = 'dirty/book'

### Books 2

The dataset consist of tableA from Goodreads and tableB from Barnes and Noble.

Metadata of the source data: The Goodreads dataset comprises 3967 samples, while the Barnes and Noble dataset contains 3701 samples. Human annotators have labeled 92 pairs as gold-standard matches. Additionally, there are 676 pairs that exhibit an exact match at the string level in the 'title' column.


The following are details to generate postive candidates (matches) and negative candidates (non-matches):

**postive candidates** : 
1. The isbnA and isbnB are the same (when existing); OR
2. (The titleA and titleB are the same) AND (The titleA is not Autobiography and An Autobiography).
3. 1100 pairs are randonly selected afterward.

**negative candidates** :
1. The idA and idB is distinct from the positive candidates; AND
2. The idA and idB is distinct from the golden candidates.
3. 1000 pairs are randonly selected afterward.

The ['Title', 'ISBN13'] columns are used for the later manual check.

In [29]:
dataset_path = 'books2/csv_files'
tableA_path = os.path.join(src_book_dir_path, dataset_path, 'goodreads.csv')
tableB_path = os.path.join(src_book_dir_path, dataset_path, 'barnes_and_noble.csv')

tableA = pd.read_csv(tableA_path)
tableB = pd.read_csv(tableB_path, encoding='latin1')

tableA.replace(r'^\s*$', np.nan, regex=True, inplace=True)
tableB.replace(r'^\s*$', np.nan, regex=True, inplace=True)

match_path = os.path.join(src_book_dir_path, dataset_path, 'labeled_data.csv')
gold_match = pd.read_csv(match_path, skiprows=5)

print(len(set(tableA.Title).intersection(set(tableB.Title))))
print(gold_match.match_label.sum())

676
92


In [30]:
# clean up
print('The old columns are: \n', tableA.columns, '\n', tableB.columns)

tableA['Authors'] = tableA.apply(lambda x: ', '.join([x[col] for col in ['FirstAuthor', 'SecondAuthor', 'ThirdAuthor'] 
                                                if pd.notnull(x[col])]), axis=1)
tableB['Authors'] = tableB.apply(lambda x: ', '.join([x[col] for col in ['Author1', 'Author2', 'Author3'] 
                                                if pd.notnull(x[col])]), axis=1)

new_tableA = tableA.drop(columns=['Description', 'ISBN', 'FirstAuthor', 'SecondAuthor', 'ThirdAuthor', 
                                  'NumberofReviews', 'Format', 'Language', 'FileName'])\
                   .rename(columns={'PageCount': 'Pages', 'NumberofRatings': 'Ratings_Count', 
                                    'PublishDate': 'Publish_Date'})
new_tableB = tableB.drop(columns=['Author1', 'Author2', 'Author3', 'Productdimensions', 'Salesrank', 
                                  'Paperbackprice', 'Hardcoverprice', 'Nookbookprice', 'Audiobookprice'])\
                   .rename(columns={'Ratingscount': 'Ratings_Count', 'Ratingvalue': 'Rating',  
                                    'PublicationDate': 'Publish_Date'})
new_tableA = new_tableA[['ID', 'Title', 'Authors', 'ISBN13', 'Pages', 'Rating', 'Ratings_Count', 'Publisher', 'Publish_Date']]
new_tableB = new_tableB[['ID', 'Title', 'Authors', 'ISBN13', 'Pages', 'Rating', 'Ratings_Count', 'Publisher', 'Publish_Date']]

print('\n The new columns are: \n', new_tableA.columns, '\n', new_tableB.columns)

# store new_tableA and new_tableB in the cleaned directory
save_dir = 'cleaned/book/Goodreads-Barnes_and_Noble'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
new_tableA.to_csv(os.path.join(save_dir, 'tableA.csv'), index=False)
new_tableB.to_csv(os.path.join(save_dir, 'tableB.csv'), index=False)

The old columns are: 
 Index(['ID', 'Title', 'Description', 'ISBN', 'ISBN13', 'PageCount',
       'FirstAuthor', 'SecondAuthor', 'ThirdAuthor', 'Rating',
       'NumberofRatings', 'NumberofReviews', 'Publisher', 'PublishDate',
       'Format', 'Language', 'FileName'],
      dtype='object') 
 Index(['ID', 'Title', 'Author1', 'Author2', 'Author3', 'Publisher', 'ISBN13',
       'PublicationDate', 'Pages', 'Productdimensions', 'Salesrank',
       'Ratingscount', 'Ratingvalue', 'Paperbackprice', 'Hardcoverprice',
       'Nookbookprice', 'Audiobookprice'],
      dtype='object')

 The new columns are: 
 Index(['ID', 'Title', 'Authors', 'ISBN13', 'Pages', 'Rating', 'Ratings_Count',
       'Publisher', 'Publish_Date'],
      dtype='object') 
 Index(['ID', 'Title', 'Authors', 'ISBN13', 'Pages', 'Rating', 'Ratings_Count',
       'Publisher', 'Publish_Date'],
      dtype='object')


In [31]:
temp_tableA = new_tableA[['ID', 'Title', 'ISBN13']].copy()
temp_tableB = new_tableB[['ID', 'Title', 'ISBN13']].copy()

connection = duckdb.connect(database=':memory:', read_only=False)
connection.register('A', temp_tableA)
connection.register('B', temp_tableB)

query = """
SELECT
    ta.ID AS ID_A,
    tb.ID AS ID_B,
    ta.Title AS Title_A,
    tb.Title AS Title_B,
    ta.ISBN13 As ISBN13_A,
    tb.ISBN13 As ISBN13_B
    
FROM
    A ta
CROSS JOIN
    B tb
WHERE (ta.ISBN13 = tb.ISBN13) OR 
    ((ta.ISBN13 IS NULL OR tb.ISBN13 IS NULL) AND (ta.Title == tb.Title) AND (ta.Title != 'Autobiography') AND
    ((ta.Title != 'An Autobiography')))
"""

result = connection.execute(query).fetchdf()
connection.close()

In [32]:
## generate pos_candidates & neg_candidates, and store them in the cleaned directory
pos_candidates = pd.DataFrame({
    'ID_A': result['ID_A'], 'ID_B': result['ID_B'],
    'Title_A': result['Title_A'], 'Title_B': result['Title_B'],
    'ISBN13_A': result['ISBN13_A'], 'ISBN13_B': result['ISBN13_B'],
})

temp_tableA['temp_key'] = 1
temp_tableB['temp_key'] = 1
all_id_pairs = pd.merge(temp_tableA[['ID', 'temp_key']], temp_tableB[['ID', 'temp_key']], on='temp_key', suffixes=('_A', '_B'))
all_id_pairs = all_id_pairs.drop('temp_key', axis=1)

gold_match = gold_match.rename(columns={'ltable.ID': 'ID_A', 'rtable.ID': 'ID_B'})
exclude_pairs = pd.concat([gold_match[['ID_A', 'ID_B']], pos_candidates[['ID_A', 'ID_B']]], axis=0).drop_duplicates()
neg_candidates = pd.concat([all_id_pairs, exclude_pairs], axis=0).drop_duplicates(keep=False)

neg_candidates = neg_candidates.sample(n=1000, random_state=SEED)
neg_candidates = neg_candidates.merge(temp_tableA[['ID', 'Title', 'ISBN13']], 
                                      left_on='ID_A', right_on='ID', how='left', suffixes=('_A', '_B')).drop(columns=['ID'])
neg_candidates = neg_candidates.merge(temp_tableB[['ID', 'Title', 'ISBN13']], 
                                      left_on='ID_B', right_on='ID', how='left', suffixes=('_A', '_B')).drop(columns=['ID'])
neg_candidates = neg_candidates[list(pos_candidates.columns)]

pos_candidates.to_csv(os.path.join(save_dir, 'pos_candidates.csv'), index=False)
neg_candidates.to_csv(os.path.join(save_dir, 'neg_candidates.csv'), index=False)

### Books 3

The dataset consist of tableA from Barnes and Noble and tableB from Half.

Metadata of the source data: The Barnes and Noble dataset comprises 3022 samples, while the Half dataset contains 3099 samples. Human annotators have labeled 327 pairs as gold-standard matches. Additionally, there are 357 pairs that exhibit an exact match at the string level in the 'title' column.


The following are details to generate postive candidates (matches) and negative candidates (non-matches):

**postive candidates** : 
1. The isbnA and isbnB are the same (when existing); OR
2. The titleA and titleB are the same.

**negative candidates** :
1. The idA and idB is distinct from the positive candidates; AND
2. The idA and idB is distinct from the golden candidates.
3. 1000 pairs are randonly selected afterward.

The ['Title', 'ISBN13'] columns are used for the later manual check.

In [98]:
dataset_path = 'books3/csv_files'
tableA_path = os.path.join(src_book_dir_path, dataset_path, 'barnes_and_noble.csv')
tableB_path = os.path.join(src_book_dir_path, dataset_path, 'half.csv')

tableA = pd.read_csv(tableA_path)
tableB = pd.read_csv(tableB_path)

tableA.replace(r'^\s*$', np.nan, regex=True, inplace=True)
tableB.replace(r'^\s*$', np.nan, regex=True, inplace=True)

match_path = os.path.join(src_book_dir_path, dataset_path, 'labeled_data.csv')
gold_match = pd.read_csv(match_path, skiprows=5)

print(len(set(tableA.Title).intersection(set(tableB.Title))))
print(gold_match.gold.sum())

357
327


In [99]:
# clean up
print('The old columns are: \n', tableA.columns, '\n', tableB.columns)

new_tableA = tableA
new_tableB = tableB.drop(columns=['UsedPrice', 'ISBN10']).rename(columns={'NewPrice': 'Price'})

new_tableA = new_tableA[['ID', 'Title', 'Author', 'Price', 'ISBN13', 'Publisher', 'Publication_Date', 'Pages', 'Dimensions']]
new_tableB = new_tableB[['ID', 'Title', 'Author', 'Price', 'ISBN13', 'Publisher', 'Publication_Date', 'Pages', 'Dimensions']]

print('\n The new columns are: \n', new_tableA.columns, '\n', new_tableB.columns)

# store new_tableA and new_tableB in the cleaned directory
save_dir = 'cleaned/book/Barnes_and_Noble-Half'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
new_tableA.to_csv(os.path.join(save_dir, 'tableA.csv'), index=False)
new_tableB.to_csv(os.path.join(save_dir, 'tableB.csv'), index=False)

The old columns are: 
 Index(['ID', 'Title', 'Price', 'Author', 'ISBN13', 'Publisher',
       'Publication_Date', 'Pages', 'Dimensions'],
      dtype='object') 
 Index(['ID', 'Title', 'UsedPrice', 'NewPrice', 'Author', 'ISBN10', 'ISBN13',
       'Publisher', 'Publication_Date', 'Pages', 'Dimensions'],
      dtype='object')

 The new columns are: 
 Index(['ID', 'Title', 'Author', 'Price', 'ISBN13', 'Publisher',
       'Publication_Date', 'Pages', 'Dimensions'],
      dtype='object') 
 Index(['ID', 'Title', 'Author', 'Price', 'ISBN13', 'Publisher',
       'Publication_Date', 'Pages', 'Dimensions'],
      dtype='object')


In [100]:
temp_tableA = new_tableA[['ID', 'Title', 'ISBN13']].copy()
temp_tableB = new_tableB[['ID', 'Title', 'ISBN13']].copy()

connection = duckdb.connect(database=':memory:', read_only=False)
connection.register('A', temp_tableA)
connection.register('B', temp_tableB)

query = """
SELECT
    ta.ID AS ID_A,
    tb.ID AS ID_B,
    ta.Title AS Title_A,
    tb.Title AS Title_B,
    ta.ISBN13 As ISBN13_A,
    tb.ISBN13 As ISBN13_B
    
FROM
    A ta
CROSS JOIN
    B tb
WHERE (ta.ISBN13 == tb.ISBN13) OR
    ((ta.ISBN13 IS NULL OR tb.ISBN13 IS NULL) AND (ta.Title == tb.Title))
"""

result = connection.execute(query).fetchdf()
connection.close()

In [39]:
## generate pos_candidates & neg_candidates, and store them in the cleaned directory
pos_candidates = pd.DataFrame({
    'ID_A': result['ID_A'], 'ID_B': result['ID_B'],
    'Title_A': result['Title_A'], 'Title_B': result['Title_B'],
    'ISBN13_A': result['ISBN13_A'], 'ISBN13_B': result['ISBN13_B'],
})

temp_tableA['temp_key'] = 1
temp_tableB['temp_key'] = 1
all_id_pairs = pd.merge(temp_tableA[['ID', 'temp_key']], temp_tableB[['ID', 'temp_key']], on='temp_key', suffixes=('_A', '_B'))
all_id_pairs = all_id_pairs.drop('temp_key', axis=1)

gold_match = gold_match.rename(columns={'ltable.ID': 'ID_A', 'rtable.ID': 'ID_B'})
exclude_pairs = pd.concat([gold_match[['ID_A', 'ID_B']], pos_candidates[['ID_A', 'ID_B']]], axis=0).drop_duplicates()
neg_candidates = pd.concat([all_id_pairs, exclude_pairs], axis=0).drop_duplicates(keep=False)

neg_candidates = neg_candidates.sample(n=1000, random_state=SEED)
neg_candidates = neg_candidates.merge(temp_tableA[['ID', 'Title', 'ISBN13']], 
                                      left_on='ID_A', right_on='ID', how='left', suffixes=('_A', '_B')).drop(columns=['ID'])
neg_candidates = neg_candidates.merge(temp_tableB[['ID', 'Title', 'ISBN13']], 
                                      left_on='ID_B', right_on='ID', how='left', suffixes=('_A', '_B')).drop(columns=['ID'])
neg_candidates = neg_candidates[list(pos_candidates.columns)]

pos_candidates.to_csv(os.path.join(save_dir, 'pos_candidates.csv'), index=False)
neg_candidates.to_csv(os.path.join(save_dir, 'neg_candidates.csv'), index=False)

### Books 5

The dataset consist of tableA from Amazon and tableB from Barnes and Noble.

Metadata of the source data: The Amazon dataset comprises 2999 samples, while the Barnes and Noble dataset contains 2998 samples. Human annotators have labeled 40 pairs as gold-standard matches. Additionally, there are 205 pairs that exhibit an exact match at the string level in the 'title' column.


The following are details to generate postive candidates (matches) and negative candidates (non-matches):

**postive candidates** : 
1. The isbnA and isbnB are the same (when existing); OR
2. The titleA contains titleB, vice verse.

**negative candidates** :
1. The idA and idB is distinct from the positive candidates; AND
2. The idA and idB is distinct from the golden candidates.
3. 1000 pairs are randonly selected afterward.

The ['Title', 'ISBN'] columns are used for the later manual check.

In [17]:
dataset_path = 'books5/csv_files'
tableA_path = os.path.join(src_book_dir_path, dataset_path, 'amazon.csv')
tableB_path = os.path.join(src_book_dir_path, dataset_path, 'barnes_and_noble.csv')

tableA = pd.read_csv(tableA_path)
tableB = pd.read_csv(tableB_path)

tableA.replace(r'^\s*$', np.nan, regex=True, inplace=True)
tableB.replace(r'^\s*$', np.nan, regex=True, inplace=True)

match_path = os.path.join(src_book_dir_path, dataset_path, 'labeled_data.csv')
gold_match = pd.read_csv(match_path, skiprows=5)

print(len(set(tableA.Title).intersection(set(tableB.title))))
print(gold_match.Gold.sum())

205
40


In [18]:
# clean up
print('The old columns are: \n', tableA.columns, '\n', tableB.columns)

tableB['authors'] = tableB['authors'].apply(lambda x: x[:-1].replace(',', ', ') if not pd.isnull(x) else x)

new_tableA = tableA.drop(columns=['ProductType', 'PublicationDate'])\
                   .rename(columns={'Author': 'Authors', 'price': 'Price'})
new_tableB = tableB.drop(columns=['cover', 'language', 'ISBN-10'])\
                   .rename(columns={'title': 'Title', 'authors': 'Authors',
                              'pages': 'Pages', 'publisher': 'Publisher',
                              'price': 'Price', 'ISBN13': 'ISBN'})

new_tableA = new_tableA[['ID', 'Title', 'Authors', 'ISBN', 'Publisher', 'Pages', 'Price']]
new_tableB = new_tableB[['ID', 'Title', 'Authors', 'ISBN', 'Publisher', 'Pages', 'Price']]

print('\n The new columns are: \n', new_tableA.columns, '\n', new_tableB.columns)

# store new_tableA and new_tableB in the cleaned directory
save_dir = 'cleaned/book/Amazon-Barnes_and_Noble'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
new_tableA.to_csv(os.path.join(save_dir, 'tableA.csv'), index=False)
new_tableB.to_csv(os.path.join(save_dir, 'tableB.csv'), index=False)

The old columns are: 
 Index(['ID', 'Title', 'Author', 'ISBN', 'Publisher', 'PublicationDate',
       'Pages', 'price', 'ProductType'],
      dtype='object') 
 Index(['ID', 'title', 'authors', 'cover', 'pages', 'publisher', 'language',
       'ISBN-10', 'ISBN13', 'price'],
      dtype='object')

 The new columns are: 
 Index(['ID', 'Title', 'Authors', 'ISBN', 'Publisher', 'Pages', 'Price'], dtype='object') 
 Index(['ID', 'Title', 'Authors', 'ISBN', 'Publisher', 'Pages', 'Price'], dtype='object')


In [19]:
temp_tableA = new_tableA[['ID', 'Title', 'ISBN']].copy()
temp_tableB = new_tableB[['ID', 'Title', 'ISBN']].copy()

connection = duckdb.connect(database=':memory:', read_only=False)
connection.register('A', temp_tableA)
connection.register('B', temp_tableB)

query = """
SELECT
    ta.ID AS ID_A,
    tb.ID AS ID_B,
    ta.Title AS Title_A,
    tb.Title AS Title_B,
    ta.ISBN As ISBN_A,
    tb.ISBN As ISBN_B
    
FROM
    A ta
CROSS JOIN
    B tb
WHERE (ta.ISBN = tb.ISBN) OR
    ((ta.ISBN IS NULL OR tb.ISBN IS NULL) AND ((LOWER(ta.Title) LIKE '%' || LOWER(tb.Title) || '%' OR LOWER(tb.Title) LIKE '%' || LOWER(ta.Title) || '%')))
"""

result = connection.execute(query).fetchdf()
connection.close()

In [20]:
result

,ID_A,ID_B,Title_A,Title_B,ISBN_A,ISBN_B
0,bn_804,B6,MySQL Crash Course,MySQL Crash Course,9780672327124,9.780672e+12
1,bn_1038,B10,Oracle8 Developer's Guide,Oracle8 Developer's Guide (Developer's Guides ...,9780764531972,9.780765e+12
2,bn_617,B15,High Availability Mysql Cookbook,High Availability MySQL Cookbook,9781847199942,9.781847e+12
3,bn_1609,B22,Automating Microsoft Access with Macros: For W...,Automating Microsoft Access With Macros: For W...,9780782118568,9.780782e+12
4,bn_276,B45,Oracle PL/SQL Programming,Oracle PL/SQL Programming: Guide to Oracle8i F...,9780596553142,NaN
...,...,...,...,...,...,...
561,bn_2570,B2881,Communicating Data with Tableau,Communicating Data with Tableau,9781449372026,9.781449e+12
562,bn_2507,B2883,Creating a Self-Tuning Oracle Database: Automa...,Creating a Self-Tuning Oracle Database: Automa...,9780972751322,9.780973e+12
563,bn_2757,B2885,Expert Oracle RAC 12c,Expert Oracle RAC 12c (The Expert's Voice),9781430250449,9.781430e+12
564,bn_2101,B2918,Expert One-on-One Microsoft Access Application...,Expert One-on-One Microsoft Access Application...,9780764559044,9.780765e+12


In [43]:
## generate pos_candidates & neg_candidates, and store them in the cleaned directory
pos_candidates = pd.DataFrame({
    'ID_A': result['ID_A'], 'ID_B': result['ID_B'],
    'Title_A': result['Title_A'], 'Title_B': result['Title_B'],
    'ISBN_A': result['ISBN_A'], 'ISBN_B': result['ISBN_B'],
})

temp_tableA['temp_key'] = 1
temp_tableB['temp_key'] = 1
all_id_pairs = pd.merge(temp_tableA[['ID', 'temp_key']], temp_tableB[['ID', 'temp_key']], on='temp_key', suffixes=('_A', '_B'))
all_id_pairs = all_id_pairs.drop('temp_key', axis=1)

gold_match = gold_match.rename(columns={'ltable.ID': 'ID_A', 'rtable.ID': 'ID_B'})
exclude_pairs = pd.concat([gold_match[['ID_A', 'ID_B']], pos_candidates[['ID_A', 'ID_B']]], axis=0).drop_duplicates()
neg_candidates = pd.concat([all_id_pairs, exclude_pairs], axis=0).drop_duplicates(keep=False)

neg_candidates = neg_candidates.sample(n=1000, random_state=SEED)
neg_candidates = neg_candidates.merge(temp_tableA[['ID', 'Title', 'ISBN']], 
                                      left_on='ID_A', right_on='ID', how='left', suffixes=('_A', '_B')).drop(columns=['ID'])
neg_candidates = neg_candidates.merge(temp_tableB[['ID', 'Title', 'ISBN']], 
                                      left_on='ID_B', right_on='ID', how='left', suffixes=('_A', '_B')).drop(columns=['ID'])
neg_candidates = neg_candidates[list(pos_candidates.columns)]

pos_candidates.to_csv(os.path.join(save_dir, 'pos_candidates.csv'), index=False)
neg_candidates.to_csv(os.path.join(save_dir, 'neg_candidates.csv'), index=False)

## Paper Domain

In [44]:
src_paper_dir_path = 'dirty/paper'

### AcadPapers

The dataset consist of tableA from arxiv and tableB from neurips.

Metadata of the source data: The arxiv dataset comprises 4000 samples, while the neurips dataset contains 4397 samples. There are 96 pairs that exhibit an exact match at the string level in the 'title' column, and no golden labeled are given.


The following are details to generate postive candidates (matches) and negative candidates (non-matches):

**postive candidates** : 
1. The titleA and titleB are the same; OR
2. The titleA and titleB are significant similar (similarity larger than 0.7, determined by their overlapping size).
3. Roughly 1100 pairs are randonly selected afterward.

**negative candidates** :
1. The idA and idB is distinct from the positive candidates; AND
2. 1000 pairs are randonly selected afterward.

The ['Title', 'Authors'] columns are used for the later manual check.

In [45]:
dataset_path = 'AcadPapers'
tableA_path = os.path.join(src_paper_dir_path, dataset_path, 'table_a.csv')
tableB_path = os.path.join(src_paper_dir_path, dataset_path, 'table_b.csv')

tableA = pd.read_csv(tableA_path)
tableB = pd.read_csv(tableB_path)

tableA.replace(r'^\s*$', np.nan, regex=True, inplace=True)
tableB.replace(r'^\s*$', np.nan, regex=True, inplace=True)

tableA
len(set(tableA.Title).intersection(set(tableB.Title)))

96

In [46]:
# clean up
print('The old columns are: \n', tableA.columns, '\n', tableB.columns)

new_tableA = tableA.drop(columns=['Abstract']).rename(columns={'_id': 'ID'})
new_tableB = tableB.drop(columns=['Abstract']).rename(columns={'_id': 'ID'})

print('\n The new columns are: \n', new_tableA.columns, '\n', new_tableB.columns)

# store new_tableA and new_tableB in the cleaned directory
save_dir = 'cleaned/paper/Arxiv-Neurips'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
new_tableA.to_csv(os.path.join(save_dir, 'tableA.csv'), index=False)
new_tableB.to_csv(os.path.join(save_dir, 'tableB.csv'), index=False)

The old columns are: 
 Index(['_id', 'Title', 'Authors', 'Date', 'Abstract', 'Journal-ref'], dtype='object') 
 Index(['_id', 'Title', 'Authors', 'Date', 'Abstract', 'Journal-ref'], dtype='object')

 The new columns are: 
 Index(['ID', 'Title', 'Authors', 'Date', 'Journal-ref'], dtype='object') 
 Index(['ID', 'Title', 'Authors', 'Date', 'Journal-ref'], dtype='object')


In [47]:
temp_tableA = new_tableA[['ID', 'Title', 'Authors']].copy()
temp_tableB = new_tableB[['ID', 'Title', 'Authors']].copy()

temp_tableA['key'] = 1
temp_tableB['key'] = 1

cross_joined_df = pd.merge(temp_tableA, temp_tableB, on='key', suffixes=('_A', '_B')).drop('key', axis=1)

In [48]:
def compute_similarity(title_a, title_b):
    set_a = set(title_a.lower().split())
    set_b = set(title_b.lower().split())
    intersection = set_a.intersection(set_b)
    smaller_set_size = min(len(set_a), len(set_b))
    if smaller_set_size > 0:
        return len(intersection) / smaller_set_size
    else:
        return 0
    
cross_joined_df['similarity'] = cross_joined_df.apply(lambda row: compute_similarity(row['Title_A'], row['Title_B']), axis=1)

In [49]:
high_sim_df = cross_joined_df[cross_joined_df['similarity']>0.7]
pos_candidates = pd.concat([high_sim_df[high_sim_df['similarity']==1],
                            high_sim_df[high_sim_df['similarity']!=1].sample(n=900, random_state=SEED)], axis=0)
pos_candidates = pos_candidates[['ID_A', 'ID_B', 'Title_A', 'Title_B', 'Authors_A', 'Authors_B']]

low_sim_df = cross_joined_df[cross_joined_df['similarity']<=0.7]
neg_candidates = low_sim_df.sample(n=1000, random_state=SEED)
neg_candidates = neg_candidates[['ID_A', 'ID_B', 'Title_A', 'Title_B', 'Authors_A', 'Authors_B']]

pos_candidates.to_csv(os.path.join(save_dir, 'pos_candidates.csv'), index=False)
neg_candidates.to_csv(os.path.join(save_dir, 'neg_candidates.csv'), index=False)

### Citations

The dataset consist of tableA from Google Scholar and tableB from DBLP.

Metadata of the source data: The Google Scholar dataset comprises 64263 samples, while the DBLP dataset contains 2616 samples. Human annotators have labeled 5347 pairs as gold-standard matches. Additionally, there are 1709 pairs that exhibit an exact match at the string level in the 'title' column.


The following are details to generate postive candidates (matches) and negative candidates (non-matches):

**postive candidates** : 
1. The titleA and titleB are the same; AND
2. The yearA and yearB are the same (when existing).
3. 1100 pairs are randonly selected afterward.

**negative candidates** :
1. The idA and idB is distinct from the positive candidates; AND
2. The idA and idB is distinct from the golden candidates.
3. 1000 pairs are randonly selected afterward.

The ['Title', 'Authors'] columns are used for the later manual check.

In [50]:
dataset_path = 'citations'
tableA_path = os.path.join(src_paper_dir_path, dataset_path, 'google_scholar.csv')
tableB_path = os.path.join(src_paper_dir_path, dataset_path, 'dblp.csv')

tableA = pd.read_csv(tableA_path)
tableB = pd.read_csv(tableB_path)

tableA.replace(r'^\s*$', np.nan, regex=True, inplace=True)
tableB.replace(r'^\s*$', np.nan, regex=True, inplace=True)

match_path = os.path.join(src_paper_dir_path, dataset_path, 'matches_dblp_scholar.csv')
gold_match = pd.read_csv(match_path)

print(len(set(tableA.title).intersection(set(tableB.title))))
print(gold_match.shape)

1709
(5347, 2)


In [51]:
# clean up
print('The old columns are: \n', tableA.columns, '\n', tableB.columns)

new_tableA = tableA.rename(columns={'id': 'ID', 'title': 'Title', 'authors': 'Authors', 
                                    'venue': 'Venue', 'year': 'Year'})

new_tableB = tableB.rename(columns={'id': 'ID', 'title': 'Title', 'authors': 'Authors', 
                                    'venue': 'Venue', 'year': 'Year'})

print('\n The new columns are: \n', new_tableA.columns, '\n', new_tableB.columns)

# store new_tableA and new_tableB in the cleaned directory
save_dir = 'cleaned/paper/Google_Scholar-DBLP'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
new_tableA.to_csv(os.path.join(save_dir, 'tableA.csv'), index=False)
new_tableB.to_csv(os.path.join(save_dir, 'tableB.csv'), index=False)

The old columns are: 
 Index(['id', 'title', 'authors', 'venue', 'year'], dtype='object') 
 Index(['id', 'title', 'authors', 'venue', 'year'], dtype='object')

 The new columns are: 
 Index(['ID', 'Title', 'Authors', 'Venue', 'Year'], dtype='object') 
 Index(['ID', 'Title', 'Authors', 'Venue', 'Year'], dtype='object')


In [52]:
temp_tableA = new_tableA[['ID', 'Title', 'Year', 'Authors']]
temp_tableB = new_tableB[['ID', 'Title', 'Year', 'Authors']]

connection = duckdb.connect(database=':memory:', read_only=False)
connection.register('A', temp_tableA)
connection.register('B', temp_tableB)

query = """
SELECT
    ta.ID AS ID_A,
    tb.ID AS ID_B,
    ta.Title AS Title_A,
    tb.Title AS Title_B,
    ta.Year AS Year_A,
    tb.Year AS Year_B,
    ta.Authors AS Authors_A,
    tb.Authors AS Authors_B
    
FROM
    A ta
CROSS JOIN
    B tb
WHERE
    (ta.Title == tb.Title) AND ((ta.Year IS NULL OR tb.Year IS NULL) OR ta.Year == tb.Year)
"""

result = connection.execute(query).fetchdf()
connection.close()

In [56]:
## generate pos_candidates & neg_candidates, and store them in the cleaned directory
pos_candidates = pd.DataFrame({
    'ID_A': result['ID_A'], 'ID_B': result['ID_B'],
    'Title_A': result['Title_A'], 'Title_B': result['Title_B'],
    'Year_A': result['Year_A'], 'Year_B': result['Year_B'],
    'Authors_A': result['Authors_A'], 'Authors_B': result['Authors_B'],
})

temp_tableA['temp_key'] = 1
temp_tableB['temp_key'] = 1
all_id_pairs = pd.merge(temp_tableA[['ID', 'temp_key']], temp_tableB[['ID', 'temp_key']], on='temp_key', suffixes=('_A', '_B'))
all_id_pairs = all_id_pairs.drop('temp_key', axis=1)

gold_match = gold_match.rename(columns={'google_scholar_id': 'ID_A', 'dblp_id': 'ID_B'})
exclude_pairs = pd.concat([gold_match[['ID_A', 'ID_B']], pos_candidates[['ID_A', 'ID_B']]], axis=0).drop_duplicates()
neg_candidates = pd.concat([all_id_pairs, exclude_pairs], axis=0).drop_duplicates(keep=False)

pos_candidates = pos_candidates.sample(n=1100, random_state=SEED)
neg_candidates = neg_candidates.sample(n=1000, random_state=SEED)

neg_candidates = neg_candidates.merge(temp_tableA[['ID', 'Title', 'Year', 'Authors']], 
                                      left_on='ID_A', right_on='ID', how='left', suffixes=('_A', '_B')).drop(columns=['ID'])
neg_candidates = neg_candidates.merge(temp_tableB[['ID', 'Title', 'Year', 'Authors']], 
                                      left_on='ID_B', right_on='ID', how='left', suffixes=('_A', '_B')).drop(columns=['ID'])
neg_candidates = neg_candidates[list(pos_candidates.columns)]

pos_candidates.to_csv(os.path.join(save_dir, 'pos_candidates.csv'), index=False)
neg_candidates.to_csv(os.path.join(save_dir, 'neg_candidates.csv'), index=False)

### CompVision

The dataset consist of tableA from arxiv and tableB from thecvf.

Metadata of the source data: The arxiv dataset comprises 3547 samples, while the thecvf dataset contains 10000 samples. There are 340 pairs that exhibit an exact match at the string level in the 'title' column, and no golden labeled are given.


The following are details to generate postive candidates (matches) and negative candidates (non-matches):

**postive candidates** : 
1. The titleA and titleB are the same; OR
2. The titleA and titleB are significant similar (similarity larger than 0.7, determined by their overlapping size).
3. Roughly 1100 pairs are randonly selected afterward.

**negative candidates** :
1. The idA and idB is distinct from the positive candidates; AND
2. 1000 pairs are randonly selected afterward.

The ['Title', 'Authors'] columns are used for the later manual check.

In [59]:
dataset_path = 'CompVision'
tableA_path = os.path.join(src_paper_dir_path, dataset_path, 'table_a.csv')
tableB_path = os.path.join(src_paper_dir_path, dataset_path, 'table_b.csv')

tableA = pd.read_csv(tableA_path)
tableB = pd.read_csv(tableB_path)

tableA.replace(r'^\s*$', np.nan, regex=True, inplace=True)
tableB.replace(r'^\s*$', np.nan, regex=True, inplace=True)

print(len(set(tableA.Title).intersection(set(tableB.Title))))

340


In [60]:
# clean up
print('The old columns are: \n', tableA.columns, '\n', tableB.columns)

new_tableA = tableA.rename(columns={'_id': 'ID'})

new_tableB = tableB.rename(columns={'_id': 'ID'})

print('\n The new columns are: \n', new_tableA.columns, '\n', new_tableB.columns)

# store new_tableA and new_tableB in the cleaned directory
save_dir = 'cleaned/paper/Arxiv-THECVF'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
new_tableA.to_csv(os.path.join(save_dir, 'tableA.csv'), index=False)
new_tableB.to_csv(os.path.join(save_dir, 'tableB.csv'), index=False)

The old columns are: 
 Index(['_id', 'Tag', 'Title', 'Authors', 'Month', 'Year', 'JournalRef'], dtype='object') 
 Index(['_id', 'Tag', 'Title', 'Authors', 'Month', 'Year', 'JournalRef'], dtype='object')

 The new columns are: 
 Index(['ID', 'Tag', 'Title', 'Authors', 'Month', 'Year', 'JournalRef'], dtype='object') 
 Index(['ID', 'Tag', 'Title', 'Authors', 'Month', 'Year', 'JournalRef'], dtype='object')


In [64]:
temp_tableA = new_tableA[['ID', 'Title', 'Authors']].copy()
temp_tableB = new_tableB[['ID', 'Title', 'Authors']].copy()

temp_tableA['key'] = 1
temp_tableB['key'] = 1

cross_joined_df = pd.merge(temp_tableA, temp_tableB, on='key', suffixes=('_A', '_B')).drop('key', axis=1)

In [65]:
def compute_similarity(title_a, title_b):
    set_a = set(title_a.lower().split())
    set_b = set(title_b.lower().split())
    intersection = set_a.intersection(set_b)
    smaller_set_size = min(len(set_a), len(set_b))
    if smaller_set_size > 0:
        return len(intersection) / smaller_set_size
    else:
        return 0
    
cross_joined_df['similarity'] = cross_joined_df.apply(lambda row: compute_similarity(row['Title_A'], row['Title_B']), axis=1)

In [68]:
high_sim_df = cross_joined_df[cross_joined_df['similarity']>0.7]
pos_candidates = pd.concat([high_sim_df[high_sim_df['similarity']==1],
                            high_sim_df[high_sim_df['similarity']!=1].sample(n=600, random_state=SEED)], axis=0)
pos_candidates = pos_candidates[['ID_A', 'ID_B', 'Title_A', 'Title_B', 'Authors_A', 'Authors_B']]

low_sim_df = cross_joined_df[cross_joined_df['similarity']<=0.7]
neg_candidates = low_sim_df.sample(n=1000, random_state=SEED)
neg_candidates = neg_candidates[['ID_A', 'ID_B', 'Title_A', 'Title_B', 'Authors_A', 'Authors_B']]

pos_candidates.to_csv(os.path.join(save_dir, 'pos_candidates.csv'), index=False)
neg_candidates.to_csv(os.path.join(save_dir, 'neg_candidates.csv'), index=False)

## Movie Domain

In [71]:
src_movie_dir_path = 'dirty/movie'

### Movies 1

The dataset consist of tableA from Rotten Tomatoes and tableB from IMDB.

Metadata of the source data: The Rotten Tomatoes dataset comprises 7390 samples, while the IMDB dataset contains 6407 samples. Human annotators have labeled 190 pairs as gold-standard matches. Additionally, there are 3466 pairs that exhibit an exact match at the string level in the 'name' column.


The following are details to generate postive candidates (matches) and negative candidates (non-matches):

**postive candidates** : 
1. The nameA and nameB are the same; AND
2. The directorA and directorB are the same.
3. 1100 pairs are randonly selected afterward.

**negative candidates** :
1. The idA and idB is distinct from the positive candidates; AND
2. The idA and idB is distinct from the golden candidates.
3. 1000 pairs are randonly selected afterward.

The ['Name', 'Director', 'Year'] columns are used for the later manual check.

In [79]:
dataset_path = 'movies1/csv_files'
tableA_path = os.path.join(src_movie_dir_path, dataset_path, 'rotten_tomatoes.csv')
tableB_path = os.path.join(src_movie_dir_path, dataset_path, 'imdb.csv')

tableA = pd.read_csv(tableA_path)
tableB = pd.read_csv(tableB_path)

tableA.replace(r'^\s*$', np.nan, regex=True, inplace=True)
tableB.replace(r'^\s*$', np.nan, regex=True, inplace=True)

match_path = os.path.join(src_movie_dir_path, dataset_path, 'labeled_data.csv')
gold_match = pd.read_csv(match_path, skiprows=5)

print(len(set(tableA.Name).intersection(set(tableB.Name))))
print(gold_match.gold.sum())

3466
190


In [80]:
# clean up
print('The old columns are: \n', tableA.columns, '\n', tableB.columns)

new_tableA = tableA.drop(columns=['Actors', 'Cast', 'Language', 'Country', 'RatingCount', 
                                  'ReviewCount', 'Filming Locations', 'Description'])\
                   .rename(columns={'Id': 'ID', 'Release Date': 'Release_Date', 'RatingValue': 'Rating'})

new_tableB = tableB.drop(columns=['Cast', 'Url', 'Description', 'ContentRating'])\
                   .rename(columns={'Id': 'ID', 'ReleaseDate': 'Release_Date', 'YearRange': 'Year',
                                    'RatingValue': 'Rating'})
new_tableA = new_tableA[['ID', 'Name', 'Director', 'Creator', 'Duration', 'Rating', 'Year', 'Release_Date', 'Genre']]
new_tableB = new_tableB[['ID', 'Name', 'Director', 'Creator', 'Duration', 'Rating', 'Year', 'Release_Date', 'Genre']]

print('\n The new columns are: \n', new_tableA.columns, '\n', new_tableB.columns)

# store new_tableA and new_tableB in the cleaned directory
save_dir = 'cleaned/movie/Rotten_Tomatoes-IMDB'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
new_tableA.to_csv(os.path.join(save_dir, 'tableA.csv'), index=False)
new_tableB.to_csv(os.path.join(save_dir, 'tableB.csv'), index=False)

The old columns are: 
 Index(['Id', 'Name', 'Year', 'Release Date', 'Director', 'Creator', 'Actors',
       'Cast', 'Language', 'Country', 'Duration', 'RatingValue', 'RatingCount',
       'ReviewCount', 'Genre', 'Filming Locations', 'Description'],
      dtype='object') 
 Index(['Id', 'Name', 'YearRange', 'ReleaseDate', 'Director', 'Creator', 'Cast',
       'Duration', 'RatingValue', 'ContentRating', 'Genre', 'Url',
       'Description'],
      dtype='object')

 The new columns are: 
 Index(['ID', 'Name', 'Director', 'Creator', 'Duration', 'Rating', 'Year',
       'Release_Date', 'Genre'],
      dtype='object') 
 Index(['ID', 'Name', 'Director', 'Creator', 'Duration', 'Rating', 'Year',
       'Release_Date', 'Genre'],
      dtype='object')


In [81]:
temp_tableA = new_tableA[['ID', 'Name', 'Director', 'Year']].copy()
temp_tableB = new_tableB[['ID', 'Name', 'Director', 'Year']].copy()

connection = duckdb.connect(database=':memory:', read_only=False)
connection.register('A', temp_tableA)
connection.register('B', temp_tableB)

query = """
SELECT
    ta.ID AS ID_A,
    tb.ID AS ID_B,
    ta.Name AS Name_A,
    tb.Name AS Name_B,
    ta.Director As Director_A,
    tb.Director As Director_B,
    ta.Year As Year_A,
    tb.Year As Year_B,
    
FROM
    A ta
CROSS JOIN
    B tb
WHERE
    ta.Name == tb.Name AND ta.Director == tb.Director
"""

result = connection.execute(query).fetchdf()
connection.close()

In [82]:
## generate pos_candidates & neg_candidates, and store them in the cleaned directory
pos_candidates = pd.DataFrame({
    'ID_A': result['ID_A'], 'ID_B': result['ID_B'],
    'Name_A': result['Name_A'], 'Name_B': result['Name_B'],
    'Director_A': result['Director_A'], 'Director_B': result['Director_B'],
    'Year_A': result['Year_A'], 'Year_B': result['Year_B'],
})

temp_tableA['temp_key'] = 1
temp_tableB['temp_key'] = 1
all_id_pairs = pd.merge(temp_tableA[['ID', 'temp_key']], temp_tableB[['ID', 'temp_key']], on='temp_key', suffixes=('_A', '_B'))
all_id_pairs = all_id_pairs.drop('temp_key', axis=1)

gold_match = gold_match.rename(columns={'ltable.Id': 'ID_A', 'rtable.Id': 'ID_B'})
exclude_pairs = pd.concat([gold_match[['ID_A', 'ID_B']], pos_candidates[['ID_A', 'ID_B']]], axis=0).drop_duplicates()
neg_candidates = pd.concat([all_id_pairs, exclude_pairs], axis=0).drop_duplicates(keep=False)

pos_candidates = pos_candidates.sample(n=1100, random_state=SEED)
neg_candidates = neg_candidates.sample(n=1000, random_state=SEED)

neg_candidates = neg_candidates.merge(temp_tableA[['ID', 'Name', 'Director', 'Year']], 
                                      left_on='ID_A', right_on='ID', how='left', suffixes=('_A', '_B')).drop(columns=['ID'])
neg_candidates = neg_candidates.merge(temp_tableB[['ID', 'Name', 'Director', 'Year']], 
                                      left_on='ID_B', right_on='ID', how='left', suffixes=('_A', '_B')).drop(columns=['ID'])
neg_candidates = neg_candidates[list(pos_candidates.columns)]

pos_candidates.to_csv(os.path.join(save_dir, 'pos_candidates.csv'), index=False)
neg_candidates.to_csv(os.path.join(save_dir, 'neg_candidates.csv'), index=False)

### Movies 4

The dataset consist of tableA from Amazon and tableB from Rotten Tomatoes.

Metadata of the source data: The Amazon dataset comprises 5241 samples, while the Rotten Tomatoes dataset contains 4392 samples. Human annotators have labeled 53 pairs as gold-standard matches. Additionally, there are 27 pairs that exhibit an exact match at the string level in the 'title' column.


The following are details to generate postive candidates (matches) and negative candidates (non-matches):

**postive candidates** : 
1. The titleA contains titleB, vice verse; AND
2. The directorA and directorB are the same.
3. 1100 pairs are randonly selected afterward.

**negative candidates** :
1. The idA and idB is distinct from the positive candidates; AND
2. The idA and idB is distinct from the golden candidates.
3. 1000 pairs are randonly selected afterward.

The ['Title', 'Director', 'Year'] columns are used for the later manual check.

In [86]:
dataset_path = 'movies4/csv_files'
tableA_path = os.path.join(src_movie_dir_path, dataset_path, 'amazon.csv')
tableB_path = os.path.join(src_movie_dir_path, dataset_path, 'rotten_tomatoes.csv')

tableA = pd.read_csv(tableA_path)
tableB = pd.read_csv(tableB_path, encoding='latin1')

tableA.replace(r'^\s*$', np.nan, regex=True, inplace=True)
tableB.replace(r'^\s*$', np.nan, regex=True, inplace=True)

match_path = os.path.join(src_movie_dir_path, dataset_path, 'labeled_data.csv')
gold_match = pd.read_csv(match_path, skiprows=5)

print(len(set(tableA.title).intersection(set(tableB.title))))
print(gold_match.gold.sum())

27
53


In [88]:
# clean up
print('The old columns are: \n', tableA.columns, '\n', tableB.columns)

tableB['cast'] = tableB.apply(lambda x: ', '.join([x[col] for col in 
                                                 ['star1', 'star2', 'star3', 'star4', 'star5', 'star6'] 
                                                 if pd.notnull(x[col])]), axis=1)

new_tableA = tableA.drop(columns=['cost'])\
                   .rename(columns={'id': 'ID', 'title': 'Title', 'time': 'Time', 'director': 'Director', 
                                    'year': 'Year', 'star': 'Cast'})

new_tableB = tableB.drop(columns=['star1', 'star2', 'star3', 'star4', 'star5', 'star6', 'review1', 'review2',
                                  'review3', 'review4', 'review5', 'rotten_tomatoes', 'audience_rating'])\
                   .rename(columns={'id': 'ID', 'title': 'Title', 'time': 'Time', 'director': 'Director', 
                                    'year': 'Year', 'cast': 'Cast'})

print('\n The new columns are: \n', new_tableA.columns, '\n', new_tableB.columns)

# store new_tableA and new_tableB in the cleaned directory
save_dir = 'cleaned/movie/Amazon-Rotten_Tomatoes'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
new_tableA.to_csv(os.path.join(save_dir, 'tableA.csv'), index=False)
new_tableB.to_csv(os.path.join(save_dir, 'tableB.csv'), index=False)

The old columns are: 
 Index(['id', 'title', 'time', 'director', 'year', 'star', 'cost'], dtype='object') 
 Index(['id', 'title', 'time', 'director', 'year', 'star1', 'star2', 'star3',
       'star4', 'star5', 'star6', 'rotten_tomatoes', 'audience_rating',
       'review1', 'review2', 'review3', 'review4', 'review5', 'cast'],
      dtype='object')

 The new columns are: 
 Index(['ID', 'Title', 'Time', 'Director', 'Year', 'Cast'], dtype='object') 
 Index(['ID', 'Title', 'Time', 'Director', 'Year', 'Cast'], dtype='object')


In [93]:
temp_tableA = new_tableA[['ID', 'Title', 'Director', 'Year']].copy()
temp_tableB = new_tableB[['ID', 'Title', 'Director', 'Year']].copy()

connection = duckdb.connect(database=':memory:', read_only=False)
connection.register('A', temp_tableA)
connection.register('B', temp_tableB)

query = """
SELECT
    ta.ID AS ID_A,
    tb.ID AS ID_B,
    ta.Title AS Title_A,
    tb.Title AS Title_B,
    ta.Director As Director_A,
    tb.Director As Director_B,
    ta.Year As Year_A,
    tb.Year As Year_B,
    
FROM
    A ta
CROSS JOIN
    B tb
WHERE 
    ((ta.Title LIKE '%' || tb.Title || '%') OR (tb.Title LIKE '%' || ta.Title || '%')) AND
    (ta.Director == tb.Director)
    
"""

result = connection.execute(query).fetchdf()
connection.close()

In [97]:
## generate pos_candidates & neg_candidates, and store them in the cleaned directory
pos_candidates = pd.DataFrame({
    'ID_A': result['ID_A'], 'ID_B': result['ID_B'],
    'Title_A': result['Title_A'], 'Title_B': result['Title_B'],
    'Director_A': result['Director_A'], 'Director_B': result['Director_B'],
    'Year_A': result['Year_A'], 'Year_B': result['Year_B'],
})

temp_tableA['temp_key'] = 1
temp_tableB['temp_key'] = 1
all_id_pairs = pd.merge(temp_tableA[['ID', 'temp_key']], temp_tableB[['ID', 'temp_key']], on='temp_key', suffixes=('_A', '_B'))
all_id_pairs = all_id_pairs.drop('temp_key', axis=1)

gold_match = gold_match.rename(columns={'ltable.id': 'ID_A', 'rtable.id': 'ID_B'})
exclude_pairs = pd.concat([gold_match[['ID_A', 'ID_B']], pos_candidates[['ID_A', 'ID_B']]], axis=0).drop_duplicates()
neg_candidates = pd.concat([all_id_pairs, exclude_pairs], axis=0).drop_duplicates(keep=False)

pos_candidates = pos_candidates.sample(n=1100, random_state=SEED)
neg_candidates = neg_candidates.sample(n=1000, random_state=SEED)

neg_candidates = neg_candidates.merge(temp_tableA[['ID', 'Title', 'Director', 'Year']], 
                                      left_on='ID_A', right_on='ID', how='left', suffixes=('_A', '_B')).drop(columns=['ID'])
neg_candidates = neg_candidates.merge(temp_tableB[['ID', 'Title', 'Director', 'Year']], 
                                      left_on='ID_B', right_on='ID', how='left', suffixes=('_A', '_B')).drop(columns=['ID'])
neg_candidates = neg_candidates[list(pos_candidates.columns)]

pos_candidates.to_csv(os.path.join(save_dir, 'pos_candidates.csv'), index=False)
neg_candidates.to_csv(os.path.join(save_dir, 'neg_candidates.csv'), index=False)

### Movies 5

The dataset consist of tableA from Roger Ebert and tableB from IMDB.

Metadata of the source data: The Roger Ebert dataset comprises 3556 samples, while the IMDB dataset contains 6913 samples. Human annotators have labeled 241 pairs as gold-standard matches. Additionally, there are 362 pairs that exhibit an exact match at the string level in the 'title' column.


The following are details to generate postive candidates (matches) and negative candidates (non-matches):

**postive candidates** : 
1. The titleA and titleB are significant similar (similarity larger than 0.7, determined by their overlapping size); AND
2. The yearA and yearB are the same (when existing).

**negative candidates** :
1. The idA and idB is distinct from the positive candidates; AND
3. 1000 pairs are randonly selected afterward.

The ['Title', 'Directors', 'Year'] columns are used for the later manual check.

In [102]:
dataset_path = 'movies5/csv_files'
tableA_path = os.path.join(src_movie_dir_path, dataset_path, 'roger_ebert.csv')
tableB_path = os.path.join(src_movie_dir_path, dataset_path, 'imdb.csv')

tableA = pd.read_csv(tableA_path)
tableB = pd.read_csv(tableB_path)

tableA.replace(r'^\s*$', np.nan, regex=True, inplace=True)
tableB.replace(r'^\s*$', np.nan, regex=True, inplace=True)

match_path = os.path.join(src_movie_dir_path, dataset_path, 'labeled_data.csv')
gold_match = pd.read_csv(match_path, skiprows=5)

print(len(set(tableA.movie_name).intersection(set(tableB.movie_name))))
print(gold_match.gold.sum())

362
241


In [103]:
# clean up
print('The old columns are: \n', tableA.columns, '\n', tableB.columns)

new_tableA = tableA.drop(columns=['pg_rating'])\
                   .rename(columns={'id': 'ID', 'movie_name': 'Title', 'year': 'Year', 'directors': 'Directors', 
                                    'actors': 'Actors', 'critic_rating': 'Content_Rating', 'genre': 'Genre',
                                    'duration': 'Duration'})

new_tableB = tableB.rename(columns={'id': 'ID', 'movie_name': 'Title', 'year': 'Year', 'directors': 'Directors', 
                                    'actors': 'Actors', 'movie_rating': 'Content_Rating', 'genre': 'Genre',
                                    'duration': 'Duration'})

print('\n The new columns are: \n', new_tableA.columns, '\n', new_tableB.columns)

# store new_tableA and new_tableB in the cleaned directory
save_dir = 'cleaned/movie/Roger_Ebert-IMDB'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
new_tableA.to_csv(os.path.join(save_dir, 'tableA.csv'), index=False)
new_tableB.to_csv(os.path.join(save_dir, 'tableB.csv'), index=False)

The old columns are: 
 Index(['id', 'movie_name', 'year', 'directors', 'actors', 'critic_rating',
       'genre', 'pg_rating', 'duration'],
      dtype='object') 
 Index(['id', 'movie_name', 'year', 'directors', 'actors', 'movie_rating',
       'genre', 'duration'],
      dtype='object')

 The new columns are: 
 Index(['ID', 'Title', 'Year', 'Directors', 'Actors', 'Content_Rating', 'Genre',
       'Duration'],
      dtype='object') 
 Index(['ID', 'Title', 'Year', 'Directors', 'Actors', 'Content_Rating', 'Genre',
       'Duration'],
      dtype='object')


In [112]:
temp_tableA = new_tableA[['ID', 'Directors', 'Title', 'Year']].copy()
temp_tableB = new_tableB[['ID', 'Directors', 'Title', 'Year']].copy()

temp_tableA['key'] = 1
temp_tableB['key'] = 1

cross_joined_df = pd.merge(temp_tableA, temp_tableB, on='key', suffixes=('_A', '_B')).drop('key', axis=1)

In [113]:
def compute_similarity(title_a, title_b):
    set_a = set(title_a.lower().split())
    set_b = set(title_b.lower().split())
    intersection = set_a.intersection(set_b)
    smaller_set_size = min(len(set_a), len(set_b))
    if smaller_set_size > 0:
        return len(intersection) / smaller_set_size
    else:
        return 0
    
cross_joined_df['similarity'] = cross_joined_df.apply(lambda row: compute_similarity(row['Title_A'], row['Title_B']), axis=1)

In [134]:
high_sim_df = cross_joined_df[cross_joined_df['similarity']>0.7]

high_sim_df = high_sim_df[(high_sim_df['Year_A']==high_sim_df['Year_B']) | (high_sim_df['Year_A'].isnull()) | (high_sim_df['Year_B'].isnull())]
pos_candidates = pd.concat([high_sim_df[(high_sim_df['similarity']==1)],
                            high_sim_df[high_sim_df['similarity']!=1]], axis=0)
pos_candidates = pos_candidates[['ID_A', 'ID_B', 'Title_A', 'Title_B', 'Directors_A', 'Directors_B', 'Year_A', 'Year_B']]

low_sim_df = cross_joined_df[cross_joined_df['similarity']<=0.7]
neg_candidates = low_sim_df.sample(n=1000, random_state=SEED)
neg_candidates = neg_candidates[['ID_A', 'ID_B', 'Title_A', 'Title_B', 'Directors_A', 'Directors_B', 'Year_A', 'Year_B']]

pos_candidates.to_csv(os.path.join(save_dir, 'pos_candidates.csv'), index=False)
neg_candidates.to_csv(os.path.join(save_dir, 'neg_candidates.csv'), index=False)

## Product Domain

In [137]:
src_product_dir_path = 'dirty/product'

### Baby Products

The dataset consist of tableA from Baby R US and tableB from Buy Buy Baby.

Metadata of the source data: The Baby R US dataset comprises 3556 samples, while the Buy Buy Baby dataset contains 6913 samples. Human annotators have labeled 108 pairs as gold-standard matches. Additionally, there are 32 pairs that exhibit an exact match at the string level in the 'title' column.


The following are details to generate postive candidates (matches) and negative candidates (non-matches):

**postive candidates** : 
1. The titleA and titleB are significant similar (similarity larger than 0.85, determined by their overlapping size).

**negative candidates** :
1. The idA and idB is distinct from the positive candidates; AND
3. 1000 pairs are randonly selected afterward.

The ['Title', 'Company', 'Price'] columns are used for the later manual check.

In [146]:
dataset_path = 'baby_products/csv_files'
tableA_path = os.path.join(src_product_dir_path, dataset_path, 'babies_r_us.csv')
tableB_path = os.path.join(src_product_dir_path, dataset_path, 'buy_buy_baby.csv')

tableA = pd.read_csv(tableA_path)
tableB = pd.read_csv(tableB_path)

tableA.replace(r'^\s*$', np.nan, regex=True, inplace=True)
tableB.replace(r'^\s*$', np.nan, regex=True, inplace=True)

match_path = os.path.join(src_product_dir_path, dataset_path, 'labeled_data.csv')
gold_match = pd.read_csv(match_path, skiprows=5)

print(len(set(tableA.title).intersection(set(tableB.title))))
print(gold_match.product_is_match.sum())

32
108


In [147]:
# clean up
print('The old columns are: \n', tableA.columns, '\n', tableB.columns)

tableA['is_discounted'] = tableA['is_discounted'].astype(bool)
tableA['dimensions'] = tableA.apply(lambda x: '/'.join([x[col] if not pd.isnull(x[col]) else '-' for col in ['length', 'width', 'height']]), axis=1)
tableB['dimensions'] = tableB.apply(lambda x: '/'.join([x[col] if not pd.isnull(x[col]) else '-' for col in ['length', 'width', 'height']]), axis=1)

new_tableA = tableA.drop(columns=['ext_id', 'company_free', 'brand', 'fabrics', 'length', 'width', 'height'])\
                   .rename(columns={'int_id': 'ID', 'title': 'Title', 'price': 'Price', 
                                    'is_discounted': 'Is_Discounted', 'category': 'Category',
                                    'company_struct': 'Company', 'weight': 'Weight', 'colors': 'Colors', 
                                    'materials': 'Materials', 'dimensions': 'Dimensions'})

new_tableB = tableB.drop(columns=['ext_id', 'company_free', 'brand', 'fabrics', 'length', 'width', 'height'])\
                   .rename(columns={'int_id': 'ID', 'title': 'Title', 'price': 'Price', 
                                    'is_discounted': 'Is_Discounted', 'category': 'Category',
                                    'company_struct': 'Company', 'weight': 'Weight', 'colors': 'Colors', 
                                    'materials': 'Materials', 'dimensions': 'Dimensions'})

print('\n The new columns are: \n', new_tableA.columns, '\n', new_tableB.columns)

# store new_tableA and new_tableB in the cleaned directory
save_dir = 'cleaned/product/Baby_U_US-Buy_Buy_Baby'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
new_tableA.to_csv(os.path.join(save_dir, 'tableA.csv'), index=False)
new_tableB.to_csv(os.path.join(save_dir, 'tableB.csv'), index=False)

The old columns are: 
 Index(['int_id', 'ext_id', 'title', 'SKU', 'price', 'is_discounted',
       'category', 'company_struct', 'company_free', 'brand', 'weight',
       'length', 'width', 'height', 'fabrics', 'colors', 'materials'],
      dtype='object') 
 Index(['int_id', 'ext_id', 'title', 'SKU', 'price', 'is_discounted',
       'category', 'company_struct', 'company_free', 'brand', 'weight',
       'length', 'width', 'height', 'fabrics', 'colors', 'materials'],
      dtype='object')

 The new columns are: 
 Index(['ID', 'Title', 'SKU', 'Price', 'Is_Discounted', 'Category', 'Company',
       'Weight', 'Colors', 'Materials', 'Dimensions'],
      dtype='object') 
 Index(['ID', 'Title', 'SKU', 'Price', 'Is_Discounted', 'Category', 'Company',
       'Weight', 'Colors', 'Materials', 'Dimensions'],
      dtype='object')


In [156]:
temp_tableA = new_tableA[['ID', 'Title', 'Company', 'Price']].copy()
temp_tableB = new_tableB[['ID', 'Title', 'Company', 'Price']].copy()

temp_tableA['temp_key'] = 1
temp_tableB['temp_key'] = 1

cross_joined_df = pd.merge(temp_tableA, temp_tableB, on='temp_key', suffixes=('_A', '_B')).drop('key', axis=1)

In [157]:
def compute_similarity(title_a, title_b):
    set_a = set(title_a.lower().split())
    set_b = set(title_b.lower().split())
    intersection = set_a.intersection(set_b)
    smaller_set_size = min(len(set_a), len(set_b))
    if smaller_set_size > 0:
        return len(intersection) / smaller_set_size
    else:
        return 0
    
cross_joined_df['similarity'] = cross_joined_df.apply(lambda row: compute_similarity(row['Title_A'], row['Title_B']), axis=1)

In [158]:
high_sim_df = cross_joined_df[cross_joined_df['similarity']>0.85]
pos_candidates = pd.concat([high_sim_df[(high_sim_df['similarity']==1)],
                            high_sim_df[high_sim_df['similarity']!=1].sample(n=900, random_state=SEED)], axis=0)
pos_candidates = pos_candidates[['ID_A', 'ID_B', 'Title_A', 'Title_B', 'Company_A', 'Company_B', 'Price_A', 'Price_B']]

low_sim_df = cross_joined_df[cross_joined_df['similarity']<=0.85]
neg_candidates = low_sim_df.sample(n=1000, random_state=SEED)
neg_candidates = neg_candidates[['ID_A', 'ID_B', 'Title_A', 'Title_B', 'Company_A', 'Company_B', 'Price_A', 'Price_B']]

pos_candidates.to_csv(os.path.join(save_dir, 'pos_candidates.csv'), index=False)
neg_candidates.to_csv(os.path.join(save_dir, 'neg_candidates.csv'), index=False)

### Cosmetics

The dataset consist of tableA from Amazon and tableB from Sephora.

Metadata of the source data: The Amazon dataset comprises 6443 samples, while the Sephora dataset contains 11026 samples. There is only 1 pair that exhibit an exact match at the string level in the 'description' column, and no golden labels are provided.


The following are details to generate postive candidates (matches) and negative candidates (non-matches):

**postive candidates** : 
1. The descriptionA contains descriptionB, vice verse; AND
2. The colorA and colorB are the same (if existing).
3. 1100 pairs are randonly selected afterward.

**negative candidates** :
1. The idA and idB is distinct from the positive candidates; AND
2. 1000 pairs are randonly selected afterward.

The ['Description', 'Color', 'Price'] columns are used for the later manual check.

In [161]:
dataset_path = 'cosmetics/csv_files'
tableA_path = os.path.join(src_product_dir_path, dataset_path, 'amazon.csv')
tableB_path = os.path.join(src_product_dir_path, dataset_path, 'sephora.csv')

tableA = pd.read_csv(tableA_path, encoding='latin1')
tableB = pd.read_csv(tableB_path)

tableA.replace(r'^\s*$', np.nan, regex=True, inplace=True)
tableB.replace(r'^\s*$', np.nan, regex=True, inplace=True)

tableA
print(len(set(tableA.Description).intersection(set(tableB.Description))))

1


In [163]:
# clean up
print('The old columns are: \n', tableA.columns, '\n', tableB.columns)

new_tableA = tableA
new_tableB = tableB.rename(columns={'Product_id': 'ID'})
new_tableA = new_tableA[['ID', 'Description', 'Price', 'Color']]
new_tableB = new_tableB[['ID', 'Description', 'Price', 'Color']]

print('\n The new columns are: \n', new_tableA.columns, '\n', new_tableB.columns)

# store new_tableA and new_tableB in the cleaned directory
save_dir = 'cleaned/product/Amazon_Sephora'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
new_tableA.to_csv(os.path.join(save_dir, 'tableA.csv'), index=False)
new_tableB.to_csv(os.path.join(save_dir, 'tableB.csv'), index=False)

The old columns are: 
 Index(['ID', 'Price', 'Color', 'Description'], dtype='object') 
 Index(['Product_id', 'Description', 'Price', 'Color'], dtype='object')

 The new columns are: 
 Index(['ID', 'Description', 'Price', 'Color'], dtype='object') 
 Index(['ID', 'Description', 'Price', 'Color'], dtype='object')


In [170]:
temp_tableA = new_tableA[['ID', 'Description', 'Color', 'Price']].copy()
temp_tableB = new_tableB[['ID', 'Description', 'Color', 'Price']].copy()

connection = duckdb.connect(database=':memory:', read_only=False)
connection.register('A', temp_tableA)
connection.register('B', temp_tableB)

query = """
SELECT
    ta.ID AS ID_A,
    tb.ID AS ID_B,
    ta.Description AS Description_A,
    tb.Description AS Description_B,
    ta.Color AS Color_A,
    tb.Color AS Color_B,
    ta.Price AS Price_A,
    tb.Price AS Price_B, 
    
FROM
    A ta
CROSS JOIN
    B tb
WHERE ((LOWER(ta.Description) LIKE '%' || LOWER(tb.Description) || '%') OR 
    (LOWER(tb.Description) LIKE '%' || LOWER(ta.Description) || '%'))
    AND ((ta.Color IS NULL OR tb.Color IS NULL) OR (ta.Color == tb.Color))  
"""


result = connection.execute(query).fetchdf()
connection.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [177]:
## generate pos_candidates & neg_candidates, and store them in the cleaned directory
pos_candidates = pd.DataFrame({
    'ID_A': result['ID_A'], 'ID_B': result['ID_B'],
    'Description_A': result['Description_A'], 'Description_B': result['Description_B'],
    'Color_A': result['Color_A'], 'Color_B': result['Color_B'],
    'Price_A': result['Price_A'], 'Price_B': result['Price_B'],
})

temp_tableA['temp_key'] = 1
temp_tableB['temp_key'] = 1
all_id_pairs = pd.merge(temp_tableA[['ID', 'temp_key']], temp_tableB[['ID', 'temp_key']], on='temp_key', suffixes=('_A', '_B'))
all_id_pairs = all_id_pairs.drop('temp_key', axis=1)


exclude_pairs = pos_candidates[['ID_A', 'ID_B']]
neg_candidates = pd.concat([all_id_pairs, exclude_pairs], axis=0).drop_duplicates(keep=False)

pos_candidates = pos_candidates.sample(n=1100, random_state=SEED)
neg_candidates = neg_candidates.sample(n=1000, random_state=SEED)

neg_candidates = neg_candidates.merge(temp_tableA[['ID', 'Description', 'Color', 'Price']], 
                                      left_on='ID_A', right_on='ID', how='left', suffixes=('_A', '_B')).drop(columns=['ID'])
neg_candidates = neg_candidates.merge(temp_tableB[['ID', 'Description', 'Color', 'Price']], 
                                      left_on='ID_B', right_on='ID', how='left', suffixes=('_A', '_B')).drop(columns=['ID'])
neg_candidates = neg_candidates[list(pos_candidates.columns)]

pos_candidates.to_csv(os.path.join(save_dir, 'pos_candidates.csv'), index=False)
neg_candidates.to_csv(os.path.join(save_dir, 'neg_candidates.csv'), index=False)

### Products

The dataset consist of tableA from Walmart and tableB from Amazon.

Metadata of the source data: The Walmart dataset comprises 2554 samples, while the Amazon dataset contains 22074 samples. Human annotators have labeled 1149 pairs as gold-standard matches. Additionally, there are 35 pairs that exhibit an exact match at the string level in the 'title' column. After several failure attempt, we decide to continue using the golden labeled samples.


The following are details to generate postive candidates (matches) and negative candidates (non-matches):

**postive candidates** : 
1. 1100 paris are randomly selected from the golden matched pairs.

**negative candidates** :
1. The idA and idB is distinct from the golden matched pairs; AND
3. 1000 pairs are randomly selected afterward.

The ['Title', 'Model_Number', 'Brand', 'Price'] columns are used for the later manual check.

In [180]:
dataset_path = 'products'
tableA_path = os.path.join(src_product_dir_path, dataset_path, 'walmart.csv')
tableB_path = os.path.join(src_product_dir_path, dataset_path, 'amazon.csv')

tableA = pd.read_csv(tableA_path)
tableB = pd.read_csv(tableB_path)

tableA.replace(r'^\s*$', np.nan, regex=True, inplace=True)
tableB.replace(r'^\s*$', np.nan, regex=True, inplace=True)

match_path = os.path.join(src_product_dir_path, dataset_path, 'matches_walmart_amazon.csv')
gold_match = pd.read_csv(match_path)

print(len(set(tableA.title).intersection(set(tableB.title))))
print(gold_match.shape)

35
(1154, 2)


In [181]:
# clean up
print('The old columns are: \n', tableA.columns, '\n', tableB.columns)

new_tableA = tableA.drop(columns=['id', 'upc', 'groupname', 'shelfdescr', 'longdescr', 'imageurl', 
                                  'orig_shelfdescr', 'orig_shortdescr', 'orig_longdescr'])\
                   .rename(columns={'custom_id': 'ID', 'brand': 'Brand', 'title': 'Title', 
                                    'price': 'Price', 'shortdescr': 'Description', 'modelno': 'Model_Number',
                                    'shipweight': 'Ship_Weight', 'dimensions': 'Dimensions'})
new_tableB = tableB.drop(columns=['url', 'asin', 'category1', 'pcategory1', 'category2', 'pcategory2',
                                  'listprice', 'prodfeatures', 'techdetails', 'proddescrlong', 'imageurl', 
                                  'itemweight', 'orig_prodfeatures', 'orig_techdetails'])\
                   .rename(columns={'custom_id': 'ID', 'brand': 'Brand', 'title': 'Title', 
                                    'price': 'Price', 'proddescrshort': 'Description', 'modelno': 'Model_Number',
                                    'shipweight': 'Ship_Weight', 'dimensions': 'Dimensions'})
new_tableA = new_tableA[['ID', 'Brand', 'Title', 'Price', 'Description', 'Model_Number', 'Ship_Weight', 'Dimensions']]
new_tableB = new_tableB[['ID', 'Brand', 'Title', 'Price', 'Description', 'Model_Number', 'Ship_Weight', 'Dimensions']]

print('\n The new columns are: \n', new_tableA.columns, '\n', new_tableB.columns)

# store new_tableA and new_tableB in the cleaned directory
save_dir = 'cleaned/product/Walmart-Amazon'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
new_tableA.to_csv(os.path.join(save_dir, 'tableA.csv'), index=False)
new_tableB.to_csv(os.path.join(save_dir, 'tableB.csv'), index=False)

The old columns are: 
 Index(['custom_id', 'id', 'upc', 'brand', 'groupname', 'title', 'price',
       'shelfdescr', 'shortdescr', 'longdescr', 'imageurl', 'orig_shelfdescr',
       'orig_shortdescr', 'orig_longdescr', 'modelno', 'shipweight',
       'dimensions'],
      dtype='object') 
 Index(['custom_id', 'url', 'asin', 'brand', 'modelno', 'category1',
       'pcategory1', 'category2', 'pcategory2', 'title', 'listprice', 'price',
       'prodfeatures', 'techdetails', 'proddescrshort', 'proddescrlong',
       'dimensions', 'imageurl', 'itemweight', 'shipweight',
       'orig_prodfeatures', 'orig_techdetails'],
      dtype='object')

 The new columns are: 
 Index(['ID', 'Brand', 'Title', 'Price', 'Description', 'Model_Number',
       'Ship_Weight', 'Dimensions'],
      dtype='object') 
 Index(['ID', 'Brand', 'Title', 'Price', 'Description', 'Model_Number',
       'Ship_Weight', 'Dimensions'],
      dtype='object')


In [194]:
temp_tableA = new_tableA[['ID', 'Title', 'Model_Number', 'Brand', 'Price']].copy()
temp_tableB = new_tableB[['ID', 'Title', 'Model_Number', 'Brand', 'Price']].copy()

temp_tableA['temp_key'] = 1
temp_tableB['temp_key'] = 1

all_id_pairs = pd.merge(temp_tableA[['ID', 'temp_key']], temp_tableB[['ID', 'temp_key']], on='temp_key', suffixes=('_A', '_B'))
all_id_pairs = all_id_pairs.drop('temp_key', axis=1)

pos_candidates = gold_match[['id1', 'id2']]
pos_candidates = pos_candidates.rename(columns={'id1': 'ID_A', 'id2': 'ID_B'})


exclude_pairs = pos_candidates[['ID_A', 'ID_B']]
neg_candidates = pd.concat([all_id_pairs, exclude_pairs], axis=0).drop_duplicates(keep=False)

pos_candidates = pos_candidates.sample(n=1100, random_state=SEED)
neg_candidates = neg_candidates.sample(n=1000, random_state=SEED)

pos_candidates = pos_candidates.merge(temp_tableA[['ID', 'Title', 'Model_Number', 'Brand', 'Price']], 
                                      left_on='ID_A', right_on='ID', how='left', suffixes=('_A', '_B')).drop(columns=['ID'])
pos_candidates = pos_candidates.merge(temp_tableB[['ID', 'Title', 'Model_Number', 'Brand', 'Price']], 
                                      left_on='ID_B', right_on='ID', how='left', suffixes=('_A', '_B')).drop(columns=['ID'])
pos_candidates = pos_candidates[['ID_A', 'ID_B', 'Title_A', 'Title_B', 'Model_Number_A', 'Model_Number_B',
                                 'Brand_A', 'Brand_B', 'Price_A', 'Price_B']]

neg_candidates = neg_candidates.merge(temp_tableA[['ID', 'Title', 'Model_Number', 'Brand', 'Price']], 
                                      left_on='ID_A', right_on='ID', how='left', suffixes=('_A', '_B')).drop(columns=['ID'])
neg_candidates = neg_candidates.merge(temp_tableB[['ID', 'Title', 'Model_Number', 'Brand', 'Price']], 
                                      left_on='ID_B', right_on='ID', how='left', suffixes=('_A', '_B')).drop(columns=['ID'])
neg_candidates = neg_candidates[list(pos_candidates.columns)]

pos_candidates.to_csv(os.path.join(save_dir, 'pos_candidates.csv'), index=False)
neg_candidates.to_csv(os.path.join(save_dir, 'neg_candidates.csv'), index=False)

# Manually Clean-UP & Create the Final Version

Upon completing the previous steps, we have created both positive and negative candidate pairs for the entity matching task. To ensure the accuracy of the new benchmark, we will perform a thorough manual review of all pairs. Once this step was finished, we can produce the final version of the datasets.

## Create the manual label column

For all neg_candidates and pos_candidates files, we will introduce a new Flag column to label the uncertain and mislabeled pairs. You can tailor your own requirement to filter out pairs you don't want. 

In [33]:
cleaned_data_dirs = {
    'restaurant': ['cleaned/restaurant/Fodors-Zagats', 
                   'cleaned/restaurant/Zomato-Yelp', 
                   'cleaned/restaurant/Yellow_Pages-Yelp'],
    
    'book': ['cleaned/book/Amazon-Barnes_and_Noble', 
             'cleaned/book/Barnes_and_Noble-Half', 
             'cleaned/book/Goodreads-Barnes_and_Noble'],
    
    'paper': ['cleaned/paper/Arxiv-Neurips', 
              'cleaned/paper/Arxiv-THECVF', 
              'cleaned/paper/Google_Scholar-DBLP'],
    
    'movie': ['cleaned/movie/Amazon-Rotten_Tomatoes', 
              'cleaned/movie/Rotten_Tomatoes-IMDB', 
              'cleaned/movie/Roger_Ebert-IMDB'],
    
    'product': ['cleaned/product/Amazon_Sephora', 
                'cleaned/product/Baby_U_US-Buy_Buy_Baby', 
                'cleaned/product/Walmart-Amazon'],
}

for domain in cleaned_data_dirs:
    for data_dir in cleaned_data_dirs[domain]:
        pos_candidates = pd.read_csv(os.path.join(data_dir, 'pos_candidates.csv'))
        neg_candidates = pd.read_csv(os.path.join(data_dir, 'neg_candidates.csv'))
        
        pos_candidates.insert(2, 'Flag', pd.NA)
        if not os.path.exists(os.path.join(data_dir, 'pos_candidates_manual.csv')):
            pos_candidates.to_csv(os.path.join(data_dir, 'pos_candidates_manual.csv'), index=False)

        neg_candidates.insert(2, 'Flag', pd.NA)
        if not os.path.exists(os.path.join(data_dir, 'neg_candidates_manual.csv')):
            neg_candidates.to_csv(os.path.join(data_dir, 'neg_candidates_manual.csv'), index=False)
        

## Create Final Datasets

To finalize the benchmark, we will first filter out data points that are manually identified as incorrect, thereby creating the golden positive and negative pairs. Subsequently, all pairs will be consolidated to form the final dataset, which will adhere to the following rules:

1. The positive pairs can consist of no more than 500 records;
2. Each dataset will comprise a total of 1000 records.

To facilitate downstream applications, we will merge Table A and Table B into a single file. Each entry will be distinguished by a suffix: '_A' for entries from Table A and '_B' for entries from Table B.

In [40]:
cleaned_data_dirs = {
    'restaurant': ['cleaned/restaurant/Fodors-Zagats', 
                   'cleaned/restaurant/Zomato-Yelp', 
                   'cleaned/restaurant/Yellow_Pages-Yelp'],
    
    'book': ['cleaned/book/Amazon-Barnes_and_Noble', 
             'cleaned/book/Barnes_and_Noble-Half', 
             'cleaned/book/Goodreads-Barnes_and_Noble'],
    
    'paper': ['cleaned/paper/Arxiv-Neurips', 
              'cleaned/paper/Arxiv-THECVF', 
              'cleaned/paper/Google_Scholar-DBLP'],
    
    'movie': ['cleaned/movie/Amazon-Rotten_Tomatoes', 
              'cleaned/movie/Rotten_Tomatoes-IMDB', 
              'cleaned/movie/Roger_Ebert-IMDB'],
    
    'product': ['cleaned/product/Amazon_Sephora', 
                'cleaned/product/Baby_U_US-Buy_Buy_Baby', 
                'cleaned/product/Walmart-Amazon'],
}

for domain in cleaned_data_dirs:
    for data_dir in cleaned_data_dirs[domain]:
        tableA = pd.read_csv(os.path.join(data_dir, 'tableA.csv'))
        tableB = pd.read_csv(os.path.join(data_dir, 'tableB.csv'))
        pos_candidates = pd.read_csv(os.path.join(data_dir, 'pos_candidates_manual.csv'))
        neg_candidates = pd.read_csv(os.path.join(data_dir, 'neg_candidates_manual.csv'))
        
        pos_pairs = pos_candidates[pos_candidates.Flag.isnull()]
        neg_pairs = neg_candidates[neg_candidates.Flag.isnull()]
        
        pos_num = min(len(pos_pairs), 500)
        neg_num = 1000 - pos_num
        
        pos_pairs = pos_pairs.sample(pos_num, random_state=SEED)
        neg_pairs = neg_pairs.sample(neg_num, random_state=SEED)
        
        pos_pairs = pos_pairs[['ID_A', 'ID_B']]
        neg_pairs = neg_pairs[['ID_A', 'ID_B']]
        
        pairs_id = pd.concat([pos_pairs, neg_pairs], axis=0)
        pairs_with_tableA = pairs_id.merge(tableA, left_on='ID_A', right_on='ID', how='left')
        pairs_with_tableA = pairs_with_tableA.drop(columns=['ID_A', 'ID_B', 'ID']).rename(columns=lambda x: x + '_A')
        pairs_with_tableB = pairs_id.merge(tableB, left_on='ID_B', right_on='ID', how='left')
        pairs_with_tableB = pairs_with_tableB.drop(columns=['ID_A', 'ID_B', 'ID']).rename(columns=lambda x: x + '_B')
        
        pairs = pd.concat([pairs_with_tableA, pairs_with_tableB], axis=1)
        pairs['Gold'] = [1] * pos_num + [0] * neg_num
        
        save_path =  '/'.join(['final'] + data_dir.split('/')[1:] + ['labeled_paris.csv'])
        if not os.path.exists(save_path):
            os.makedirs(os.path.dirname(save_path))
        pairs.to_csv(save_path, index=False)